First we'll use some Jupyter notebook magic to import the code from the
previous notebooks.

In [1]:
%%capture
%run Register\ File\ Example.ipynb
%run Defining\ an\ APB\ Model\ in\ Python.ipynb

Now we'll use the [fault](https://github.com/leonardt/fault) Python package to
verify our register file generator.  Fault provides an object `Tester` that
accepts a magma circuit and allows the user to record test actions and run them
using various target simulators.  Fault's primary goal is to provide an
interface for general purpose hardware verification in Python, allowing design
teams to perform end-to-end verification in the same language environment as
the design input (magma) rather than operating on the verilog code generated by
the magma compiler.

Fault uses a staged meta-programming architecture where the test action
sequences are recorded, but not executed, in Python.  After the test action
sequence has been recorded, the user compiles the test bench to a target
language (`C++` or `system-verilog`) and runs the simulation using a target
simulator (`verilator` for `C++`, `ncsim` or `vcs` for `system-verilog`).  This
abstraction allows the user easily port test bench code to various simulation
environments.

First, we will test performing a simple write to a register using the APB
interface.  We begin by defining a list of registers and generating a concrete
register file circuit.

In [2]:
print(RegFile)

RegFile_reg0_reg1(apb: Tuple(PCLK=In(Clock),PRESETn=In(Reset),PADDR=In(Bits[1]),PPROT=In(Bit),PENABLE=In(Bit),PWRITE=In(Bit),PWDATA=In(Bits[32]),PSTRB=In(Bits[4]),PREADY=Out(Bit),PRDATA=Out(Bits[32]),PSLVERR=Out(Bit),PSEL1=In(Bit)), reg0_d: In(Bits[32]), reg0_en: In(Enable), reg0_q: Out(Bits[32]), reg1_d: In(Bits[32]), reg1_q: Out(Bits[32]))


Now we use the generated `RegFile` to instantiate a `fault.Tester` object.

In [3]:
import fault
tester = fault.Tester(RegFile, clock=RegFile.apb.PCLK)

Using the tester object, we can record various actions. We can "poke" the
APB reset signal using Python's attribute syntax.  To access a port on the
test circuit, we use `texter.circuit`.  Setting an attribute on the circuit
corresponds to recording an action that sets the port to a specific value.

This line sets the `PRESETn` field of the `apb` port on the circuit

In [4]:
tester.circuit.apb.PRESETn = 1

In [5]:
print(tester)

Actions:
    0: Poke(RegFile_reg0_reg1.apb.PRESETn, 1)



Now we will instantiate a few objects from our `APBModel` to generate the 
low-level actions for our tester.  We use an `APBBus` to model an APB master
and make a write request to one of the registers.

In [6]:
addr_width = 1
data_width = 32
addr = 1
data = 45
bus = APBBus(addr_width, data_width, num_slaves=2)

Let's define a helper method `make_request` which provides a convenient
interface for constructing instances of `APBBusIO` and `Request`.

In [7]:
from hwtypes import BitVector


def make_request(addr, data, addr_width, data_width, num_slaves=1, slave_id=0):
    request = Request(addr_width, data_width, num_slaves)(
        APBCommand.IDLE, BitVector[addr_width](addr),
        BitVector[data_width](data),
        BitVector[max(math.ceil(math.log2(num_slaves)), 1)](slave_id))

    # Specialized instance of APB for addr/data width
    _APB = APB(addr_width, data_width, num_slaves)

    io = APBBusIO(addr_width, data_width,
                  num_slaves)(default_APB_instance(_APB), request)
    return io, request

io, request = make_request(addr, data, addr_width, data_width, num_slaves=2,
                           slave_id=1)

Finally we'll define a method `write` which interacts with the APB model
(`bus`) to generate the low-level signal `poke` values that are recorded in
`tester`.

In [8]:
from hwtypes import Bit


def set_apb_inputs(tester, bus):
    for key in tester._circuit.apb.Ks:
        # Skip clock signals
        if key in ["PCLK", "PRESETn"]:
            continue
        if tester._circuit.apb[key].isoutput():
            setattr(tester.circuit.apb, key,
                    getattr(bus.io.apb, key))


def step(bus, io, tester):
    bus(io)
    set_apb_inputs(tester, bus)
    tester.step(2)


def write(bus, io, request, tester, addr, data):
    # Test idle state
    step(bus, io, tester)

    # Send request
    request.command = APBCommand.WRITE
    step(bus, io, tester)

    request.command = APBCommand.IDLE

    # No wait state
    io.apb.PREADY = Bit(1)
    step(bus, io, tester)

    tester.circuit.apb.PREADY.expect(1)

    step(bus, io, tester)

    tester.circuit.apb.PREADY.expect(0)
    tester.step(2)

write(bus, io, request, tester, addr, data)

Let's print out the tester object to see the action sequence that has been
recorded.

In [9]:
print(tester)

Actions:
    0: Poke(RegFile_reg0_reg1.apb.PRESETn, 1)
    1: Poke(RegFile_reg0_reg1.apb.PADDR, 0)
    2: Poke(RegFile_reg0_reg1.apb.PPROT, 0)
    3: Poke(RegFile_reg0_reg1.apb.PENABLE, 0)
    4: Poke(RegFile_reg0_reg1.apb.PWRITE, 0)
    5: Poke(RegFile_reg0_reg1.apb.PWDATA, 0)
    6: Poke(RegFile_reg0_reg1.apb.PSTRB, 0)
    7: Poke(RegFile_reg0_reg1.apb.PSEL1, 0)
    8: Step(RegFile_reg0_reg1.apb.PCLK, steps=2)
    9: Poke(RegFile_reg0_reg1.apb.PADDR, 1)
    10: Poke(RegFile_reg0_reg1.apb.PPROT, 0)
    11: Poke(RegFile_reg0_reg1.apb.PENABLE, 0)
    12: Poke(RegFile_reg0_reg1.apb.PWRITE, 1)
    13: Poke(RegFile_reg0_reg1.apb.PWDATA, 45)
    14: Poke(RegFile_reg0_reg1.apb.PSTRB, 0)
    15: Poke(RegFile_reg0_reg1.apb.PSEL1, 1)
    16: Step(RegFile_reg0_reg1.apb.PCLK, steps=2)
    17: Poke(RegFile_reg0_reg1.apb.PADDR, 1)
    18: Poke(RegFile_reg0_reg1.apb.PPROT, 0)
    19: Poke(RegFile_reg0_reg1.apb.PENABLE, 1)
    20: Poke(RegFile_reg0_reg1.apb.PWRITE, 1)
    21: Poke(RegFile_reg0_reg1.a

Then, to verify that the write occured succesfully, we expect that the register
at address `addr` (fetched by name) is producing the write value on its output
port.

In [10]:
getattr(tester.circuit, f"reg{addr}_q").expect(data)

Finally, we can compile and run out test bench using `verilator`.

In [11]:
tester.compile_and_run(target="verilator", magma_output="coreir-verilog",
                       flags=["-Wno-UNUSED"])

Calling `compile_and_run` resulted in zero output, which means the test passed.
We can turn on logging to see more information

In [12]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

tester.compile_and_run(target="verilator", magma_output="coreir-verilog",
                       flags=["-Wno-UNUSED"])

DEBUG:root:verilator -Wall -Wno-INCABSPATH -Wno-DECLFILENAME -Wno-UNUSED --cc RegFile_reg0_reg1.v  --exe RegFile_reg0_reg1_driver.cpp --top-module RegFile_reg0_reg1
INFO:root:Running tester...
DEBUG:root:make -C obj_dir -j -f VRegFile_reg0_reg1.mk VRegFile_reg0_reg1
DEBUG:root:clang++  -I.  -MMD -I/usr/local/Cellar/verilator/4.010/share/verilator/include -I/usr/local/Cellar/verilator/4.010/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fbracket-depth=4096 -Qunused-arguments -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -Wno-shadow       -c -o RegFile_reg0_reg1_driver.o ../RegFile_reg0_reg1_driver.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VRegFile_reg0_reg1.cpp > VRegFile_reg0_reg1__ALLcls.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=inc

DEBUG:root:clang++  -I.  -MMD -I/usr/local/Cellar/verilator/4.010/share/verilator/include -I/usr/local/Cellar/verilator/4.010/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fbracket-depth=4096 -Qunused-arguments -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -Wno-shadow       -c -o RegFile_reg0_reg1_driver.o ../RegFile_reg0_reg1_driver.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VRegFile_reg0_reg1.cpp > VRegFile_reg0_reg1__ALLcls.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VRegFile_reg0_reg1__Syms.cpp > VRegFile_reg0_reg1__ALLsup.cpp
clang++  -I.  -MMD -I/usr/local/Cellar/verilator/4.010/share/verilator/include -I/usr/local/Cellar/verilator/4.010/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRAC

Let's write a more complex test using the write and read requests

In [13]:
def read(bus, io, request, tester, addr, data):
    # Send request
    request.command = APBCommand.READ
    step(bus, io, tester)

    request.command = APBCommand.IDLE

    # No wait state
    io.apb.PREADY = Bit(1)
    step(bus, io, tester)

    tester.circuit.apb.PREADY.expect(1)
    tester.circuit.apb.PRDATA.expect(data)
    step(bus, io, tester)

    tester.circuit.apb.PREADY.expect(0)
    tester.step(2)

tester.clear()
tester.circuit.apb.PRESETn = 1

io, request = make_request(addr, data, addr_width, data_width, num_slaves=2,
                           slave_id=1)

write(bus, io, request, tester, addr, data)

read(bus, io, request, tester, addr, data)

tester.compile_and_run(target="verilator", magma_output="coreir-verilog",
                       flags=["-Wno-UNUSED"])

DEBUG:root:verilator -Wall -Wno-INCABSPATH -Wno-DECLFILENAME -Wno-UNUSED --cc RegFile_reg0_reg1.v  --exe RegFile_reg0_reg1_driver.cpp --top-module RegFile_reg0_reg1
INFO:root:Running tester...
DEBUG:root:make -C obj_dir -j -f VRegFile_reg0_reg1.mk VRegFile_reg0_reg1
DEBUG:root:clang++  -I.  -MMD -I/usr/local/Cellar/verilator/4.010/share/verilator/include -I/usr/local/Cellar/verilator/4.010/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fbracket-depth=4096 -Qunused-arguments -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -Wno-shadow       -c -o RegFile_reg0_reg1_driver.o ../RegFile_reg0_reg1_driver.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VRegFile_reg0_reg1.cpp > VRegFile_reg0_reg1__ALLcls.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=inc

DEBUG:root:clang++  -I.  -MMD -I/usr/local/Cellar/verilator/4.010/share/verilator/include -I/usr/local/Cellar/verilator/4.010/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fbracket-depth=4096 -Qunused-arguments -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -Wno-shadow       -c -o RegFile_reg0_reg1_driver.o ../RegFile_reg0_reg1_driver.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VRegFile_reg0_reg1.cpp > VRegFile_reg0_reg1__ALLcls.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VRegFile_reg0_reg1__Syms.cpp > VRegFile_reg0_reg1__ALLsup.cpp
clang++  -I.  -MMD -I/usr/local/Cellar/verilator/4.010/share/verilator/include -I/usr/local/Cellar/verilator/4.010/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRAC

Finally, we'll issue a sequence of writes, followed by a sequence of reads

In [14]:
tester.clear()
tester.circuit.apb.PRESETn = 1

values = ((0xDE, 0xAD), (0xBE, 0xEF))
for i in range(2):
    for addr, data in enumerate(values[i]):
        io, request = make_request(addr, data, addr_width, data_width,
                                   num_slaves=2, slave_id=1)
        write(bus, io, request, tester, addr, data)
        getattr(tester.circuit, f"reg{addr}_q").expect(data)

    for addr, data in enumerate(values[i]):
        io, request = make_request(addr, data, addr_width, data_width,
                                   num_slaves=2, slave_id=1)
        read(bus, io, request, tester, addr, data)

tester.compile_and_run(target="verilator", magma_output="coreir-verilog",
                       flags=["-Wno-UNUSED"])

DEBUG:root:verilator -Wall -Wno-INCABSPATH -Wno-DECLFILENAME -Wno-UNUSED --cc RegFile_reg0_reg1.v  --exe RegFile_reg0_reg1_driver.cpp --top-module RegFile_reg0_reg1
INFO:root:Running tester...
DEBUG:root:make -C obj_dir -j -f VRegFile_reg0_reg1.mk VRegFile_reg0_reg1
DEBUG:root:clang++  -I.  -MMD -I/usr/local/Cellar/verilator/4.010/share/verilator/include -I/usr/local/Cellar/verilator/4.010/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fbracket-depth=4096 -Qunused-arguments -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -Wno-shadow       -c -o RegFile_reg0_reg1_driver.o ../RegFile_reg0_reg1_driver.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VRegFile_reg0_reg1.cpp > VRegFile_reg0_reg1__ALLcls.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=inc

DEBUG:root:clang++  -I.  -MMD -I/usr/local/Cellar/verilator/4.010/share/verilator/include -I/usr/local/Cellar/verilator/4.010/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRACE=0 -faligned-new -fbracket-depth=4096 -Qunused-arguments -Wno-parentheses-equality -Wno-sign-compare -Wno-uninitialized -Wno-unused-parameter -Wno-unused-variable -Wno-shadow       -c -o RegFile_reg0_reg1_driver.o ../RegFile_reg0_reg1_driver.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VRegFile_reg0_reg1.cpp > VRegFile_reg0_reg1__ALLcls.cpp
/usr/bin/perl /usr/local/Cellar/verilator/4.010/share/verilator/bin/verilator_includer -DVL_INCLUDE_OPT=include VRegFile_reg0_reg1__Syms.cpp > VRegFile_reg0_reg1__ALLsup.cpp
clang++  -I.  -MMD -I/usr/local/Cellar/verilator/4.010/share/verilator/include -I/usr/local/Cellar/verilator/4.010/share/verilator/include/vltstd -DVL_PRINTF=printf -DVM_COVERAGE=0 -DVM_SC=0 -DVM_TRAC